# ComfyUI for Google Colab - 安定版

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chi-saku/comfyui-paperspace-スターター/blob/main/ComfyUI_for_Colab.ipynb)

## 使い方
1. 下のセルを実行
2. 表示されるCloudflare URLをクリック
3. ComfyUIで画像生成開始！

In [ ]:
from google.colab import drive
import os, sys, time, re, socket, subprocess, threading

COMFY_DIR = "/content/drive/MyDrive/ComfyUI"
PORT = 8123
MODEL_URL = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors"
MODEL_PATH = f"{COMFY_DIR}/models/checkpoints/v1-5-pruned-emaonly.safetensors"

def run_cmd(cmd, check=True, shell=False):
    print("$ " + (cmd if isinstance(cmd, str) else " ".join(cmd)))
    return subprocess.run(cmd, check=check, shell=shell)

print("■ Step-1: Googleドライブをマウントします...")
drive.mount("/content/drive", force_remount=False)

print("\n■ Step-2: 既存プロセスをクリーンアップします...")
subprocess.run('pkill -f cloudflared || true', shell=True)
subprocess.run(f'fuser -k {PORT}/tcp || true', shell=True)
time.sleep(2)

print("\n■ Step-3: ComfyUIとManagerをクリーンインストールします...")
if os.path.exists(COMFY_DIR):
    run_cmd(['rm', '-rf', COMFY_DIR])
run_cmd(['git', 'clone', '--depth=1', 'https://github.com/comfyanonymous/ComfyUI.git', COMFY_DIR])
run_cmd(['git', 'clone', '--depth=1', 'https://github.com/ltdrdata/ComfyUI-Manager.git', f'{COMFY_DIR}/custom_nodes/ComfyUI-Manager'])

print("\n■ Step-4: 依存関係をインストールします...")
run_cmd([sys.executable, '-m', 'pip', 'install', '-q', '--upgrade', 'pip', 'setuptools', 'wheel'])
run_cmd([sys.executable, '-m', 'pip', 'install', '-q', 'torch', 'torchvision', 'torchaudio', '--index-url', 'https://download.pytorch.org/whl/cu121'])
try:
    run_cmd([sys.executable, '-m', 'pip', 'install', '-q', 'xformers'])
except subprocess.CalledProcessError:
    print("⚠️ xformersのインストールに失敗しましたが続行します")
run_cmd([sys.executable, '-m', 'pip', 'install', '-q', '-r', f'{COMFY_DIR}/requirements.txt'])
try:
    run_cmd([sys.executable, '-m', 'pip', 'install', '-q', '-r', f'{COMFY_DIR}/custom_nodes/ComfyUI-Manager/requirements.txt'])
except subprocess.CalledProcessError:
    print("⚠️ Manager要件の一部でエラーが出ましたが続行します")

print("\n■ Step-5: 基本モデルをダウンロードします...")
os.makedirs(f"{COMFY_DIR}/models/checkpoints", exist_ok=True)
if not os.path.exists(MODEL_PATH):
    run_cmd(['wget', '-c', MODEL_URL, '-O', MODEL_PATH])
else:
    print("✅ 既にモデルがあります")

print("\n■ Step-6: cloudflared を準備します...")
if not os.path.exists('/usr/local/bin/cloudflared'):
    run_cmd(['wget', '-q', 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64', '-O', '/usr/local/bin/cloudflared'])
    run_cmd(['chmod', '+x', '/usr/local/bin/cloudflared'])
else:
    print("✅ cloudflared は既にあります")

def wait_for_port(host, port, timeout=180):
    print(f"🔍 {host}:{port} が開くのを待機中...")
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection((host, port), timeout=1):
                print("✅ ポートが開きました")
                return True
        except Exception:
            time.sleep(1)
    print("❌ タイムアウト: ポートが開きませんでした")
    return False

def start_comfyui():
    print("\n🚀 ComfyUI を起動します...")
    env = os.environ.copy()
    env["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
    os.chdir(COMFY_DIR)
    cmd = [
        sys.executable, 'main.py',
        '--listen=0.0.0.0',
        f'--port={PORT}',
        f'--output-directory={COMFY_DIR}/output',
        '--disable-cuda-malloc',
        '--normalvram'
    ]
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
    ready_keywords = ('Uvicorn running', 'Application startup complete', f'http://0.0.0.0:{PORT}')
    for line in iter(proc.stdout.readline, ''):
        if not line:
            break
        if any(k.lower() in line.lower() for k in ready_keywords):
            print("🎉 ComfyUI が起動しました")
            break
    return proc

def start_cloudflared():
    print("\n🌐 cloudflared トンネルを作成します（HTTP/2）...")
    cmd = [
        'cloudflared', 'tunnel',
        f'--url=http://0.0.0.0:{PORT}',
        '--protocol', 'http2',
        '--no-autoupdate',
        '--loglevel', 'warn'
    ]
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
    public_url = None
    for line in iter(proc.stdout.readline, ''):
        if not line:
            break
        print(f"[cloudflared] {line.strip()}")
        m = re.search(r'https://[a-z0-9-]+\\.trycloudflare\\.com', line)
        if m:
            public_url = m.group(0)
            print(f"\n🚀 公開URL: {public_url}")
            print("📱 上記URLでComfyUIにアクセスできます")
            break
    return proc, public_url

comfy_proc = start_comfyui()
if wait_for_port("127.0.0.1", PORT, timeout=180):
    cf_proc, url = start_cloudflared()
    if not url:
        print("⚠️ トンネルURLを取得できませんでした（ログを確認）")
    print("\n==============================================")
    print("✅ 準備完了。セッションを維持します（停止はランタイム切断）")
    print("==============================================")
    try:
        while True:
            time.sleep(30)
            if comfy_proc.poll() is not None:
                print("⚠️ ComfyUIプロセスが終了しました")
                break
            if cf_proc.poll() is not None:
                print("⚠️ cloudflaredプロセスが終了しました")
                break
    except KeyboardInterrupt:
        print("\n🛑 停止要求を受けました。終了します。")
else:
    print("❌ ComfyUI の起動に失敗しました")